In [ ]:
pip install langchain-google-genai

In [ ]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash" ,api_key=GEMINI_API_KEY)


In [ ]:
# We will scrape hospital website and different routes using multing threading
# Analyse the data with `gemini-1.5-flash` and get the json
from concurrent.futures import ThreadPoolExecutor
import requests
from bs4 import BeautifulSoup

def getTextFromHtml(soup):
        strips = list(soup.stripped_strings)

        # Connect all pieces of textxsinto a single string
        string = ''
        for s in strips:
            string += ' ' + s

        print(string)
        return string

def scrapeAndGetTextInWebsite(url):
        try:
            # Fetch the webpage content
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}
            response = requests.get(url, headers=headers)
            # response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            string = getTextFromHtml(soup)
            return string
        except requests.exceptions.RequestException as e:
            print(f"Error fetching URL: {e}")
            return ""  # Return an empty dictionary in case of a request error
        except Exception as e:
            print(f"An error occurred: {e}")
            return ""

def getResultFromLLM(text):
        res = llm.invoke(f"""
    I have scraped the website and you are given the data.
    See if you can fetch name, address or location, phone number, and i want doctors list of information in a list like if you can see the doctor information i want all that doctor information
    from the scraped data that I give you and if you find any relavent data that is important give that also
    This is the scraped data: {text}
    Provide the information in JSON format if you did not find any data the send null in that perticular
                        """)
        school_details = {}
        res.content=str(res.content.replace("json"," "))
        res.content=str(res.content.replace("```"," "))

        print(res.content)
        jsonData={}
        try:
          jsonData=json.loads(res.content)
        except Exception as e:
          print(e)
        return jsonData

# def getTextAndAddToMainText():


def dataFromListOfUrls(listOfUrls):
  with ThreadPoolExecutor(max_workers=10) as executor:
      results = list(executor.map(scrapeAndGetTextInWebsite, listOfUrls))
      results =" ".join(results)
  return results

def mainFunction(url):
    allText=""

    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        # print(len([a['href'] for a in soup.find_all('a', href=True)]))

        # links = {a['href'] for a in soup.find_all('a', href=True) if '#' not in a['href']}
        links = {
                  a['href']
                  for a in soup.find_all('a', href=True)
                  if a['href'] != '#' and a['href'] != ''
                }

        string = scrapeAndGetTextInWebsite(url)
        allText =allText+ " "+string

        otherLinks=[]
        if(url.endswith("/")):
          url=url.removesuffix("/")
        dumyLinks=[]
        for i in links:
          if(i.startswith("/")):
            dumyLinks.append(url+i)
            # i=url+i
          else:
            dumyLinks.append(i)


        dumyText= dataFromListOfUrls(dumyLinks)
        allText = allText + dumyText
        school_details=getResultFromLLM(allText)

        return school_details

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return {}  # Return an empty dictionary in case of a request error
    except Exception as e:
        print(f"An error occurred: {e}")
        return {}

# Example usage
url = "https://madhavbaug.org/"
details = mainFunction(url)
print(details)


In [ ]:
# Sample Output for above:
{
  "doctors": [
    {
      "name": "Dr Gurudatta Amin",
      "title": "Chief Medical Officer",
      "additional_info": null
    },
    {
      "name": "Dr. Pravin Ghadigaonkar",
      "title": "Head Of Medical Operations",
      "additional_info": null
    },
    {
      "name": "Dr Vidyut Ghag",
      "title": "Director and Chief Administrative Officer",
      "additional_info": null
    },
    {
      "name": "Dr. Nilesh Kulthe",
      "title": "Zonal Medical Head",
      "additional_info": null
    },
    {
      "name": "Dr. Bipin Gond",
      "title": "Zonal Medical Head",
      "additional_info": null
    },
    {
      "name": "Dr. Rohit Sane",
      "title": "Founder & Chief Executive Officer (CEO)",
      "additional_info": "Managing Director of the Company. MBBS (Medicine and Surgery) and fellowship in cardiac Rehabilitation, over 20 years of health care consulting."
    }

  ],
  "locations": [
    {
      "name": "Khopoli Hospital",
      "address": null,
      "phone": null
    },
    {
      "name": "kondhali hospital",
      "address": "Survey No - 2, Mauja Salai (Khurd), Khursapar, Nagpur Amravati Highway, Near Kondhali, Tal - Katol, Dist - Nagpur - 441 103.",
      "phone": "+91 77964 53155"
    },
    {
      "name": "vizag hospital",
      "address": "CLSPL, Plot No. 11, Mercury Avenue, District 4,\nAMTZ Campus, Pragathi Maidan, VM Steel Plant,\nVisakhapatnam, Andhra Pradesh - 530 031.",
      "phone": "040452 09200"
    },
        {
      "name": "Madhavbaug (Thane)",
      "address": "1st Floor, Nandgaonkar Heights, Uthalsar Rd, Uthalsar Naka, Uthalsar, Thane West, Maharashtra 400601",
      "phone": null
    }
  ],
  "contact": {
    "phone": [
      "040 4520 9200",
      "+91 70218 07630",
      "+91 70218 07650",
      "madhavbaug.vizag@gmail.com"
    ],
    "email": [
      "info@madhavbaug.org",
      "madhavbaug.vizag@gmail.com",
      "madbavbaughospitalkondhali@gmail.com",
      "itsupport@madhavbaug.com",
      "investors@madhavbaug.com",
      "cs@madhavbaug.com",
      "grc.madhavbaug@gmail.com",
      "sanecare.madhavbaug@gmail.com"

    ],
    "address": "1st Floor, Nandgaonkar Heights, Uthalsar Rd, Uthalsar Naka, Uthalsar, Thane West, Maharashtra 400601"
  }
}